In [1]:
import os
from dotenv import load_dotenv
from py2neo import Graph
load_dotenv()
password = os.getenv('DBPASS')

graph = Graph("bolt://localhost:7687", auth=("neo4j", password))

In [193]:
query = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton), 
    (p)-[:ELECTED_TO]->(ch:Chamber)

RETURN i.name AS issue, 
    c.abbrev AS canton, 
    ch.name AS chamber,
    COUNT(DISTINCT(b)) AS num_bills,
    (20 + substring(toString(b.bill_number), 0, 2)) AS year

ORDER BY issue, canton, year;
"""

num_bills_result = graph.run(query).data()

In [194]:
import json

with open('mps_per_canton.json', 'r') as file:
    adjustment_data = json.load(file)

In [195]:
consolidated_data = {}

for entry in num_bills_result:
    issue = entry["issue"]
    canton = entry["canton"]
    year = entry["year"]
    chamber = entry["chamber"]
    num_bills = entry["num_bills"]

    key = (issue, canton, year)

    if key not in consolidated_data:
        consolidated_data[key] = {
            "issue": issue,
            "canton": canton,
            "year": year,
            "num_bills_national_adj": 0,
            "num_bills_state_adj": 0
        }

    if chamber == "Nationalrat":
        adjustment = adjustment_data["National_council"].get(canton, 1)
        consolidated_data[key]["num_bills_national_adj"] += num_bills / adjustment
        consolidated_data[key]["num_bills_national"] = num_bills
    elif chamber == "Ständerat":
        adjustment = adjustment_data["Council_of_states"].get(canton, 1)
        consolidated_data[key]["num_bills_state_adj"] += num_bills / adjustment
        consolidated_data[key]["num_bills_state"] = num_bills

num_bills_adjusted_result = list(consolidated_data.values())

# Add weight
for d in num_bills_adjusted_result:
    d["num_bills_adj"] = (d["num_bills_national_adj"] + d["num_bills_state_adj"]) / 2

In [200]:
query = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton),
      (pg: Party)<-[:MEMBER_OF]-(p)
WITH i.name AS issue,
     c.abbrev AS canton,
     (20 + substring(toString(b.bill_number), 0, 2)) AS year,
     pg.name AS parl_group,
     b
UNWIND labels(b) AS bill_label
WITH issue, canton, year, parl_group, bill_label, COUNT(DISTINCT b) AS num_bills
WHERE bill_label <> 'Bill' 
WITH issue, canton, year, parl_group, bill_label, num_bills
RETURN issue, 
       canton, 
       year, 
       collect({group: parl_group, count: num_bills, label: bill_label}) AS group_data
ORDER BY issue, canton, year;
"""

parl_group_result = graph.run(query).data()

In [201]:
query = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton)
WITH i.name AS issue, 
     c.abbrev AS canton, 
     (20 + substring(toString(b.bill_number), 0, 2)) AS year, 
     b
UNWIND labels(b) AS bill_label
WITH issue, canton, year, bill_label, b
WHERE bill_label <> 'Bill'
WITH issue, canton, year, bill_label, COUNT(DISTINCT b) AS num_bills
RETURN issue, 
       canton, 
       year, 
       collect({label: bill_label, count: num_bills}) AS bill_data
ORDER BY issue, canton, year;
"""

bill_labels_result = graph.run(query).data()

In [202]:
query = """
MATCH (i:Issuelist)<-[:ABOUT]-(b:Bill)<-[:SPONSORS]-(p:Person)-[:REPRESENTS]->(c:Canton)
WITH i.name AS issue, 
     c.abbrev AS canton, 
     (20 + substring(toString(b.bill_number), 0, 2)) AS year,
     p.uid AS id,
     p.first_name AS first_name, 
     p.last_name AS last_name,
     p.gender AS gender,
     p.native_language AS native_language,
     COUNT(DISTINCT b) AS num_bills
ORDER BY issue, canton, year, num_bills DESC
WITH issue, canton, year, COLLECT({id: id, first_name: first_name, last_name: last_name, count: num_bills, gender: gender, native_language: native_language}) AS top_persons
RETURN issue, 
       canton, 
       year, 
       top_persons
ORDER BY issue, canton, year;
"""

bill_persons_result = graph.run(query).data()

In [203]:
from collections import defaultdict

def join_dicts_on_keys(lists, keys):
    joined_dict = defaultdict(dict)
    
    for lst in lists:
        for d in lst:
            key = tuple(d[k] for k in keys)
            values_without_keys = {k: v for k, v in d.items() if k not in keys}
            joined_dict[key].update(values_without_keys)
    
    return [dict(**dict(zip(keys, key)), **values) for key, values in joined_dict.items()]

result = join_dicts_on_keys([num_bills_adjusted_result, parl_group_result, bill_labels_result, bill_persons_result], keys=["canton", "year", "issue"])

In [204]:
with open('data.json', 'w') as file:
    json.dump(result, file, indent=4)